# updated on 5/25/2019

# Notebook Version

##### Functions that go into download_prep.py

In [3]:
# import os
# import sys
# import subprocess
# import time
# from stat import S_ISDIR, ST_CTIME, ST_MODE
# from shutil import copytree, rmtree
# import pause
# from selenium.webdriver import Chrome
# from selenium.webdriver.chrome import webdriver as chrome_webdriver
# from selenium.webdriver.support.ui import Select

# # relative import to grab user_creds
# sys.path.append('../../user_creds/')
# import account_info as acc_info

# class DriverBuilder():
#     # https://stackoverflow.com/questions/45631715/downloading-with-chrome-headless-and-selenium
#     def get_driver(self, download_location=None, headless=False):
#         driver = self._get_chrome_driver(download_location, headless)
#         driver.set_window_size(1400, 700)
#         return driver

#     def _get_chrome_driver(self, download_location, headless):
#         chrome_options = chrome_webdriver.Options()
#         if download_location:
#             prefs = {'download.default_directory': download_location,
#                      'download.prompt_for_download': False,
#                      'download.directory_upgrade': True,
#                      'safebrowsing.enabled': False,
#                      'safebrowsing.disable_download_protection': True}
#             chrome_options.add_experimental_option('prefs', prefs)
#             chrome_options.add_argument("--no-sandbox")

#         if headless:
#             chrome_options.add_argument("--headless")

#         driver_path = '/usr/bin/chromedriver'

#         if sys.platform.startswith("win"):
#             driver_path += ".exe"

#         driver = Chrome(executable_path=driver_path,
#                         chrome_options=chrome_options)
#         if headless:
#             self.enable_download_in_headless_chrome(driver, download_location)
#         return driver

#     def enable_download_in_headless_chrome(self, driver, download_dir):
#         """
#         there is currently a "feature" in chrome where
#         headless does not allow file download: https://bugs.chromium.org/p/chromium/issues/detail?id=696481
#         This method is a hacky work-around until the official chromedriver support for this.
#         Requires chrome version 62.0.3196.0 or above.
#         """

#         # add missing support for chrome "send_command"  to selenium webdriver
#         driver.command_executor._commands["send_command"] = (
#             "POST", '/session/$sessionId/chromium/send_command')

#         params = {'cmd': 'Page.setDownloadBehavior', 'params': {
#             'behavior': 'allow', 'downloadPath': download_dir}}
#         command_result = driver.execute("send_command", params)
#         print("response from browser:")
#         for key in command_result:
#             print("result:" + key + ":" + str(command_result[key]))

# def download_csvs(download_path):
#     '''
#     downloads all loan_info csvs and pmt_history csv
#     '''
#     # setup constants
#     email = acc_info.email_throwaway
#     password = acc_info.password_throwaway
#     url_dl = "https://www.lendingclub.com/info/download-data.action"
#     url_signin = "https://www.lendingclub.com/auth/login"
#     url_pmt_hist = "https://www.lendingclub.com/site/additional-statistics"

#     d_builder = DriverBuilder()
#     driver = d_builder.get_driver(
#         download_location=download_path, headless=True)
    
#     # sign in
#     driver.get(url_signin)
#     email_box = driver.find_element_by_xpath(
#         '/html/body/div[2]/div[1]/div[2]/form[1]/label[1]/input')
#     password_box = driver.find_element_by_xpath(
#         '/html/body/div[2]/div[1]/div[2]/form[1]/label[2]/input')

#     pause.milliseconds(1000)
#     email_box.send_keys(email)
#     password_box.send_keys(password)

#     button = driver.find_element_by_xpath(
#         '/html/body/div[2]/div[1]/div[2]/form[1]/button')
#     button.click()
#     pause.milliseconds(3000)

#     # download loan_info
#     driver.get(url_dl)
#     download_btn = driver.find_element_by_xpath(
#         '//*[@id="currentLoanStatsFileName"]')

#     select = driver.find_element_by_xpath(
#         '//*[@id="loanStatsDropdown"]')  # get the select element
#     options = select.find_elements_by_tag_name(
#         "option")  # get all the options into a list

#     options_dict = {}
#     for option in options:  # iterate over the options, place attribute value in list
#         options_dict[option.get_attribute("value")] = option.text

#     for opt_val, text in options_dict.items():
#         print("starting download on option {0}, {1}".format(opt_val, text))

#         select = driver.find_element_by_xpath(
#             '//*[@id="loanStatsDropdown"]')
#         selection = Select(select)
#         selection.select_by_value(opt_val)
#         download_btn.click()
#         pause.milliseconds(2000)

#     # payment history downloads
#     driver.get(url_pmt_hist)

#     pmt_history = driver.find_element_by_xpath(
#         '/html/body/div[2]/section/div[2]/div/p[2]/a[2]')
#     pmt_history.click()

#     # wait for all downloads to finish
#     while True:
#         if len(os.listdir(download_path)) != (
#                 len(options_dict) + 1):  # +1 for one pmt history file
#             time.sleep(5)
#             print('waiting for all csv downloads to start')
#             continue
#         else:
#             files = os.listdir(download_path)
#             k = 0
#             time.sleep(5)
#             print('checking/waiting for all csv downloads to finish')
#             for filename in files:
#                 if 'crdownload' in filename:
#                     print('{0} is still downloading'.format(filename))
#                     time.sleep(30)
#                 else:
#                     k += 1
#     #                 print(k)
#             if k == len(files):
#                 time.sleep(2)
#                 break

#     print('done downloading')
#     driver.close()
#     return True

# def get_hashes(path):
#     hashes = {}
#     files = os.listdir(path)
#     for file_ in files:
#         a = subprocess.check_output(
#             'shasum -a 256 {0}'.format(path + '/' + file_), shell=True)
#         hashes[file_] = a.split()[0]
#     return hashes

# def check_file_changes(csv_folders, just_dled_hashes):
#     need_to_clean = False
#     try:
#         previous_dled_hashes = get_hashes(csv_folders[-2][1])

#         # compare new download to previous download
#         # check for added or deleted files
#         dne_files = set(just_dled_hashes.keys()).intersection(
#             set(previous_dled_hashes.keys()))
#         add_files = set(previous_dled_hashes.keys()).intersection(
#             set(just_dled_hashes.keys()))
#         if len(just_dled_hashes) != len(previous_dled_hashes):
#             need_to_clean = True
#             print("Compared to the previous time new csv's were downloaded, the following files were deleted: \n {0}".format(
#                 dne_files))
#             print("Compared to the previous time new csv's were downloaded, the following files are new additions: \n {0}".format(
#                 add_files))
#         else:
#             print('No files were added or deleted since previous downloading of csvs')

#         # check for shasum256 changes
#         changed_files = []
#         for key in just_dled_hashes.keys() & previous_dled_hashes.keys():
#             if previous_dled_hashes[key] != just_dled_hashes[key]:
#                     changed_files.append(key)
                
#         if len(changed_files) == 0:
#             print('There are no changes to previous downloaded lending club csvs (loan_info and pmt_hist) via shasum256 hashes')
#         else:
#             need_to_clean = True
#             print('Compared to the previous data download, the shasum256 hashes changed for the following files: {0}'.format(
#                 changed_files))

#     except IndexError:
#         need_to_clean = True
#         print('Could not find previously download directory? This is probably your first time downloading the csvs or the first download to a new path.')

#     return need_to_clean

# def get_sorted_creationtime_dirs(ppath):
#     csv_folders = [os.path.join(ppath, fn) for fn in os.listdir(
#         ppath) if fn not in ['archived_csvs', 'working_csvs']]
#     csv_folders = [(os.stat(path), path) for path in csv_folders]
#     csv_folders = [(stat[ST_CTIME], path)
#                    for stat, path in csv_folders if S_ISDIR(stat[ST_MODE])]
#     return sorted(csv_folders)

# def archiver(archive_flag, ppath, archiver_dir=None):
#     archiver_dir = os.path.join(os.path.expanduser(
#         '~'), 'projects', 'lendingclub', 'data', 'csvs', 'archived_csvs') if not archiver_dir else archiver_dir
#     os.makedirs(archiver_dir, exist_ok=True)
#     if archive_flag:
#         just_dled = get_sorted_creationtime_dirs(ppath)[-1][1]
        
#         newest_folder = os.path.split(just_dled)[1]
        
#         copytree(just_dled, os.path.join(archiver_dir, newest_folder))
        
#         print('copied {0} to {1}'.format(newest_folder, archiver_dir))

# def cleaner(ppath):
#     just_dled = os.path.split(get_sorted_creationtime_dirs(ppath)[-1][1])[1]
#     keep_dirs = ['archived_csvs', 'working_csvs', just_dled]
#     for tree in os.listdir(ppath):
#         if tree not in keep_dirs:
#             rmtree(os.path.join(ppath, tree))
#             print('removing old dirs {0}'.format(tree))
#     return just_dled

##### notebook version

In [4]:
# # setup
# rel_path = '../../data/csvs'
# now = time.strftime("%m_%d_%Hh_%Mm_%Ss_%Y")
# download_dir = 'csvs_' + now
# os.mkdir(download_dir)

# # download csvs
# download_csvs(download_dir)

# # calculate shasum256 hash on just downloaded csvs
# just_dled_hashes = get_hashes(download_dir)

# # copy the downloaded files to data location
# copytree(download_dir, os.path.join(rel_path, download_dir))

# # delete the original
# rmtree(download_dir)

# # get the dirs holding downloaded csvs by creation time (not archived_csvs dir)
# csv_folders = get_sorted_creationtime_dirs(rel_path)

# # check if compared to previous time, there are changes/additions/deletions in csvs
# archive_flag = check_file_changes(csv_folders, just_dled_hashes)

# # if something was different (archive_flag), then store a copy of just_downloaded to archives
# archiver(archive_flag, rel_path)

# # removes old downloads
# cleaner(rel_path)

# print('done downloading, checking, and archiving (when necessary) the csv files!!!')

# loadscript for editing

In [21]:
%%writefile ../../lendingclub/csv_dl_archiving/download_prep.py
"""
This module contains functions that assist in downloading and archiving csvs from
lendingclub (loan info and payment history). Selenium is used to download the files.
Other helper functions take care of comparing to any previously downloaded csvs
and archiving when necessary.

TODOS:
selenium can sometimes go too fast and miss some of the download options
I currently get around this by explicitly pausing for 2 seconds near the
problem areas, but should probably create a function to check downloading csvs
with all options in dropdown list.
"""
# %load ../../lendingclub/csv_dl_archiving/download_prep.py
# driver download https://github.com/mozilla/geckodriver/releases
# extracted geckodriver to /usr/local/bin in ubuntu
import os
import stat
from stat import S_ISDIR, ST_CTIME, ST_MODE
import subprocess
import sys
import time
from datetime import datetime
from shutil import copytree, rmtree

import pause
from selenium.webdriver import Chrome
from selenium.webdriver.chrome import webdriver as chrome_webdriver
from selenium.webdriver.support.ui import Select
import user_creds.account_info as acc_info
from lendingclub import config


# import to grab user_creds
sys.path.append(config.prj_dir)



class DriverBuilder():
    '''
    helps build chrome driver
    https://stackoverflow.com/questions/45631715/downloading-with-chrome-headless-and-selenium
    '''
    def get_driver(self, download_location=None, headless=False):
        '''
        helps get driver
        '''
        driver = self._get_chrome_driver(download_location, headless)
        driver.set_window_size(1400, 700)
        return driver

    def _get_chrome_driver(self, download_location, headless):
        '''
        Makes the driver with right options
        '''
        chrome_options = chrome_webdriver.Options()
        if download_location:
            prefs = {
                'download.default_directory': download_location,
                'download.prompt_for_download': False,
                'download.directory_upgrade': True,
                'safebrowsing.enabled': False,
                'safebrowsing.disable_download_protection': True
            }
            chrome_options.add_experimental_option('prefs', prefs)
            chrome_options.add_argument("--no-sandbox")

        if headless:
            chrome_options.add_argument("--headless")

        driver_path = '/usr/bin/chromedriver'

        if sys.platform.startswith("win"):
            driver_path += ".exe"

        driver = Chrome(executable_path=driver_path,
                        chrome_options=chrome_options)
        if headless:
            self.enable_download_in_headless_chrome(driver, download_location)
        return driver

    def enable_download_in_headless_chrome(self, driver, download_dir):
        """
        there is currently a "feature" in chrome where
        headless does not allow file download:
        https://bugs.chromium.org/p/chromium/issues/detail?id=696481
        This method is a hacky work-around until the official chromedriver support for this.
        Requires chrome version 62.0.3196.0 or above.
        """

        # add missing support for chrome "send_command"  to selenium webdriver
        driver.command_executor._commands["send_command"] = (
            "POST", '/session/$sessionId/chromium/send_command')

        params = {
            'cmd': 'Page.setDownloadBehavior',
            'params': {
                'behavior': 'allow',
                'downloadPath': download_dir
            }
        }
        command_result = driver.execute("send_command", params)
        print("response from browser:")
        for key in command_result:
            print("result:" + key + ":" + str(command_result[key]))


def download_csvs(download_path):
    '''
    downloads all loan_info csvs and pmt_history csv
    '''
    print('downloading csvs to {0}'.format(os.path.abspath(download_path)))

    try:
        # setup constants
        email = acc_info.email_throwaway
        password = acc_info.password_throwaway
        url_dl = "https://www.lendingclub.com/info/download-data.action"
        url_signin = "https://www.lendingclub.com/auth/login"
        url_pmt_hist = "https://www.lendingclub.com/site/additional-statistics"

        d_builder = DriverBuilder()
        driver = d_builder.get_driver(download_location=download_path,
                                      headless=True)

        # sign in
        driver.get(url_signin)
        pause.milliseconds(3000)
        email_box = driver.find_element_by_name('email')
        password_box = driver.find_element_by_name('password')
        # password_box = driver.find_element_by_xpath(
        #     '/html/body/div[2]/div[1]/div[2]/form[1]/label[2]/input')

        pause.milliseconds(1000)
        email_box.send_keys(email)
        password_box.send_keys(password)

        # button = driver.find_element_by_xpath(
        #     '/html/body/div[2]/div[1]/div[2]/form[1]/button')
        button = driver.find_element_by_class_name('form-button--submit')
        button.click()
        pause.milliseconds(3000)

        # download loan_info
        driver.get(url_dl)
        # download_btn = driver.find_element_by_xpath(
        #     '//*[@id="currentLoanStatsFileName"]')
        download_btn = driver.find_element_by_id(
            'currentLoanStatsFileNameHandler')

        select = driver.find_element_by_xpath(
            '//*[@id="loanStatsDropdown"]')  # get the select element
        options = select.find_elements_by_tag_name(
            "option")  # get all the options into a list

        options_dict = {}
        for option in options:  # iterate over the options, place attribute value in list
            options_dict[option.get_attribute("value")] = option.text

        for opt_val, text in options_dict.items():
            print("starting download on option {0}, {1}".format(opt_val, text))

            select = driver.find_element_by_xpath(
                '//*[@id="loanStatsDropdown"]')
            # print('found the select dropdown')
            selection = Select(select)
            selection.select_by_value(opt_val)
            # print('set selection to option value {0}: {1}'.format(opt_val, text))
            download_btn.click()
            pause.milliseconds(2000)
            # print('got to click download btn')
            # now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
            # driver.get_screenshot_as_file('screenshot-%s.png'%now)
            # setup to workaround the center-block button:
            try:
                blocking_btn = driver.find_element_by_class_name(
                    'center-block')
                blocking_btn.click()
                close_btn = driver.find_element_by_class_name('close')
                close_btn.click()
                # print('got through blocking data disclaimer')
            except:
                # print('did not get through blocking data disclaimer')
                pass
            pause.milliseconds(2000)

        # payment history downloads
        driver.get(url_pmt_hist)

        pmt_history = driver.find_element_by_xpath(
            '/html/body/div[2]/section/div[2]/div/p[2]/a[2]')
        pmt_history.click()

        # wait for all downloads to finish
        while True:
            if len(os.listdir(download_path)) != (
                    len(options_dict) + 1):  # +1 for one pmt history file
                time.sleep(5)
                print('waiting for all csv downloads to start')
                continue
            else:
                files = os.listdir(download_path)
                k = 0
                time.sleep(5)
                print('checking/waiting for all csv downloads to finish')
                for filename in files:
                    if 'crdownload' in filename:
                        print('{0} is still downloading'.format(filename))
                        time.sleep(30)
                    else:
                        k += 1
                        #                 print(k)
                if k == len(files):
                    time.sleep(2)
                    break

        print('done downloading')
        driver.close()
        return True
    except Exception as e:
        print(e)
        now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        driver.get_screenshot_as_file('screenshot-%s.png' % now)


def get_hashes(path):
    '''
    gets shasum hashes for files to check for file changes
    '''
    print('computing shasum for files in {0}'.format(os.path.abspath(path)))
    hashes = {}
    files = os.listdir(path)
    for file_ in files:
        sha = subprocess.check_output('shasum -a 256 {0}\
                                      '.format(path + '/'
                                               + file_),
                                      shell=True)
        hashes[file_] = sha.split()[0]
    return hashes


def check_file_changes(csv_folders, just_dled_hashes):
    '''
    Checks for sha differences for downloaded vs archived csvs
    '''
    need_to_clean = False
    print(
        'starting to check for file changes comparing what was just downloaded.'
    )
    print('looking for previous downloads at {0}'.format(csv_folders))
    try:
        previous_dled_hashes = get_hashes(csv_folders[-2][1])

        # compare new download to previous download
        # check for added or deleted files
        dne_files = set(just_dled_hashes.keys()).intersection(
            set(previous_dled_hashes.keys()))
        add_files = set(previous_dled_hashes.keys()).intersection(
            set(just_dled_hashes.keys()))
        if len(just_dled_hashes) != len(previous_dled_hashes):
            need_to_clean = True
            print("Compared to the previous time new csv's were downloaded, \
                      the following files were deleted: \n {0}".format(
                          dne_files))
            print("Compared to the previous time new csv's were downloaded, \
                      the following files are new additions: \n {0}".format(
                          add_files))
        else:
            print(
                'No files were added or deleted since previous downloading of csvs'
            )

        # check for shasum256 changes
        changed_files = []
        for key in just_dled_hashes.keys() & previous_dled_hashes.keys():
            if previous_dled_hashes[key] != just_dled_hashes[key]:
                changed_files.append(key)

        if len(changed_files) == 0:
            print('There are no changes to previous downloaded lending club \
                      csvs (loan_info and pmt_hist) via shasum256 hashes')
        else:
            need_to_clean = True
            print('Compared to the previous data download, the shasum256 \
            hashes changed for the following files: {0}'.format(changed_files))

    except IndexError:
        need_to_clean = True
        print('Could not find previously download directory? This is \
                  probably your first time downloading the csvs or the \
                  first download to a new path.')

    return need_to_clean


def get_sorted_creationtime_dirs(ppath):
    '''Sort dirs by creation time'''
    print('getting folders sorted by creation time in {0}'.format(
        os.path.abspath(ppath)))
    csv_folders = [
        os.path.join(ppath, fn) for fn in os.listdir(ppath)
        if fn not in ['archived_csvs', 'working_csvs', 'latest_csvs']
    ]
    csv_folders = [(os.stat(path), path) for path in csv_folders]
    csv_folders = [(stat[ST_CTIME], path) for stat, path in csv_folders
                   if S_ISDIR(stat[ST_MODE])]
    return sorted(csv_folders)


def archiver(archive_flag, ppath, archiver_dir=None):
    '''will archive dirs if told to'''
    archiver_dir = os.path.join(
        os.path.expanduser('~'), 'projects', 'lendingclub', 'data', 'csvs',
        'archived_csvs') if not archiver_dir else archiver_dir
    os.makedirs(archiver_dir, exist_ok=True)
    if archive_flag:
        just_dled = get_sorted_creationtime_dirs(ppath)[-1][1]

        newest_folder = os.path.split(just_dled)[1]

        copytree(just_dled, os.path.join(archiver_dir, newest_folder))

        print('copied {0} to {1}'.format(newest_folder,
                                         os.path.abspath(archiver_dir)))


def cleaner(ppath):
    '''
    cleans the ppath of every dir except archived_csvs
    '''
    just_dled = os.path.split(get_sorted_creationtime_dirs(ppath)[-1][1])[1]
    keep_dirs = ['archived_csvs']  # , 'working_csvs', just_dled]
    for tree in os.listdir(ppath):
        if tree not in keep_dirs:
            rmtree(os.path.join(ppath, tree), onerror=handle_error)
            print('removing old dirs {0}'.format(tree))
    return just_dled




def handle_error(func, path, exc_info):
    '''
    Error handler function
    It will try to change file permission and call the calling function again,
    From https://thispointer.com/python-how-to-delete-a-directory-recursively-using-shutil-rmtree/
    '''
    print('Handling Error for file ', path)
    print(exc_info)
    # Check if file access issue
    if not os.access(path, os.W_OK):
        print('Hello')
        # Try to change the permision of file
        os.chmod(path, stat.S_IWUSR)
        # call the calling function again
        func(path)

Overwriting ../../lendingclub/csv_dl_archiving/download_prep.py


# writing script

In [21]:
%%writefile ../../lendingclub/csv_dl_archiving/01_download_LC_csvs.py
'''
Downloads LC csvs into dir
'''
# %load ../../lendingclub/csv_dl_archiving/01_download_LC_csvs.py
import os
import time
from shutil import copytree, rmtree

from lendingclub import config
from lendingclub.csv_dl_archiving import download_prep as dp

# setup

now = time.strftime("%m_%d_%Hh_%Mm_%Ss_%Y")
arch_name = 'raw_zipped_csvs_'+now

# make archive if it doesn't exist
os.makedirs(config.arch_dir, exist_ok=True)

# if dir for new downloads exists, delete it
if os.path.isdir(config.raw_dl_dir):
    rmtree(config.raw_dl_dir)

# download csvs
dp.download_csvs(config.raw_dl_dir)

# calculate shasum256 hash on just downloaded csvs
just_dled_hashes = dp.get_hashes(config.raw_dl_dir)


if len(os.listdir(config.arch_dir)) == 0:
    print("Archives of csvs are empty. Moving just downloaded into archives")
    copytree(config.raw_dl_dir, os.path.join(config.arch_dir, arch_name))
else:
    compare_dir = dp.get_newest_creationtime_dir(config.arch_dir)

    # check if compared to previous time, there are changes/additions/deletions in csvs
    archive_flag = dp.check_file_changes(compare_dir, just_dled_hashes)

    if archive_flag:
        print('Downloads differ from most recent archived csvs. \
              Copy to archives')
        copytree(config.raw_dl_dir, os.path.join(config.arch_dir, arch_name))
    else:
        print('Downloads do not differ from last archived. Not archiving')

print('done downloading, checking, and archiving (when necessary) the csv files!!!')



#     # if something was different (archive_flag), then store a copy of just_downloaded to archives
#     dp.archiver(archive_flag, config.csv_dir, archiver_dir = config.arch_dir)

    # # removes old downloads
    # dp.cleaner(config.csv_dir)


# # copy the downloaded files to data location
# copytree(download_dir, os.path.join(config.csv_dir, download_dir))

# # delete the original
# rmtree(download_dir)

Overwriting ../../lendingclub/csv_dl_archiving/01_download_LC_csvs.py


In [19]:
%%writefile ../../lendingclub/csv_dl_archiving/download_prep.py
"""
This module contains functions that assist in downloading and archiving csvs from
lendingclub (loan info and payment history). Selenium is used to download the files.
Other helper functions take care of comparing to any previously downloaded csvs
and archiving when necessary.

TODOS:
selenium can sometimes go too fast and miss some of the download options
I currently get around this by explicitly pausing for 2 seconds near the
problem areas, but should probably create a function to check downloading csvs
with all options in dropdown list.
"""
# %load ../../lendingclub/csv_dl_archiving/download_prep.py
# driver download https://github.com/mozilla/geckodriver/releases
# extracted geckodriver to /usr/local/bin in ubuntu
import os
# import stat
from stat import S_ISDIR, ST_CTIME, ST_MODE
import subprocess
import sys
import time
from datetime import datetime
# from shutil import copytree, rmtree

import pause
from selenium.webdriver import Chrome
from selenium.webdriver.chrome import webdriver as chrome_webdriver
from selenium.webdriver.support.ui import Select
import user_creds.account_info as acc_info
from lendingclub import config


# import to grab user_creds
sys.path.append(config.prj_dir)



class DriverBuilder():
    '''
    helps build chrome driver
    https://stackoverflow.com/questions/45631715/downloading-with-chrome-headless-and-selenium
    '''
    def get_driver(self, download_location=None, headless=False):
        '''
        helps get driver
        '''
        driver = self._get_chrome_driver(download_location, headless)
        driver.set_window_size(1400, 700)
        return driver

    def _get_chrome_driver(self, download_location, headless):
        '''
        Makes the driver with right options
        '''
        chrome_options = chrome_webdriver.Options()
        if download_location:
            prefs = {
                'download.default_directory': download_location,
                'download.prompt_for_download': False,
                'download.directory_upgrade': True,
                'safebrowsing.enabled': False,
                'safebrowsing.disable_download_protection': True
            }
            chrome_options.add_experimental_option('prefs', prefs)
            chrome_options.add_argument("--no-sandbox")

        if headless:
            chrome_options.add_argument("--headless")

        driver_path = '/usr/bin/chromedriver'

        if sys.platform.startswith("win"):
            driver_path += ".exe"

        driver = Chrome(executable_path=driver_path,
                        chrome_options=chrome_options)
        if headless:
            self.enable_download_in_headless_chrome(driver, download_location)
        return driver

    def enable_download_in_headless_chrome(self, driver, download_dir):
        """
        there is currently a "feature" in chrome where
        headless does not allow file download:
        https://bugs.chromium.org/p/chromium/issues/detail?id=696481
        This method is a hacky work-around until the official chromedriver support for this.
        Requires chrome version 62.0.3196.0 or above.
        """

        # add missing support for chrome "send_command"  to selenium webdriver
        driver.command_executor._commands["send_command"] = (
            "POST", '/session/$sessionId/chromium/send_command')

        params = {
            'cmd': 'Page.setDownloadBehavior',
            'params': {
                'behavior': 'allow',
                'downloadPath': download_dir
            }
        }
        command_result = driver.execute("send_command", params)
        print("response from browser:")
        for key in command_result:
            print("result:" + key + ":" + str(command_result[key]))


def download_csvs(download_path, pause_len=2500):
    '''
    downloads all loan_info csvs and pmt_history csv
    '''
    print('downloading csvs to {0}'.format(os.path.abspath(download_path)))

    try:
        # setup constants
        email = acc_info.email_throwaway
        password = acc_info.password_throwaway
        url_dl = "https://www.lendingclub.com/info/download-data.action"
        url_signin = "https://www.lendingclub.com/auth/login"
        url_pmt_hist = "https://www.lendingclub.com/site/additional-statistics"

        d_builder = DriverBuilder()
        driver = d_builder.get_driver(download_location=download_path,
                                      headless=True)

        # sign in
        driver.get(url_signin)
        pause.milliseconds(pause_len)
        email_box = driver.find_element_by_name('email')
        password_box = driver.find_element_by_name('password')
        # password_box = driver.find_element_by_xpath(
        #     '/html/body/div[2]/div[1]/div[2]/form[1]/label[2]/input')

        pause.milliseconds(pause_len)
        email_box.send_keys(email)
        password_box.send_keys(password)

        # button = driver.find_element_by_xpath(
        #     '/html/body/div[2]/div[1]/div[2]/form[1]/button')
        button = driver.find_element_by_class_name('form-button--submit')
        button.click()
        pause.milliseconds(pause_len)

        # download loan_info
        driver.get(url_dl)
        # download_btn = driver.find_element_by_xpath(
        #     '//*[@id="currentLoanStatsFileName"]')
        download_btn = driver.find_element_by_id(
            'currentLoanStatsFileNameHandler')

        select = driver.find_element_by_xpath(
            '//*[@id="loanStatsDropdown"]')  # get the select element
        options = select.find_elements_by_tag_name(
            "option")  # get all the options into a list

        options_dict = {}
        for option in options:  # iterate over the options, place attribute value in list
            options_dict[option.get_attribute("value")] = option.text

        for opt_val, text in options_dict.items():
            print("starting download on option {0}, {1}".format(opt_val, text))

            select = driver.find_element_by_xpath(
                '//*[@id="loanStatsDropdown"]')
            # print('found the select dropdown')
            selection = Select(select)
            selection.select_by_value(opt_val)
            # print('set selection to option value {0}: {1}'.format(opt_val, text))
            download_btn.click()
            pause.milliseconds(pause_len)
            # print('got to click download btn')
            # now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
            # driver.get_screenshot_as_file('screenshot-%s.png'%now)
            # setup to workaround the center-block button:
            try:
                blocking_btn = driver.find_element_by_class_name(
                    'center-block')
                blocking_btn.click()
                close_btn = driver.find_element_by_class_name('close')
                close_btn.click()
                # print('got through blocking data disclaimer')
            except:
                # print('did not get through blocking data disclaimer')
                pass
            pause.milliseconds(pause_len)

        # payment history downloads
        pause.milliseconds(pause_len)
        driver.get(url_pmt_hist)

        pause.milliseconds(pause_len)
        pmt_history = driver.find_element_by_partial_link_text('All payments')
        pmt_history.click()

        # wait for all downloads to finish
        while True:
            if len(os.listdir(download_path)) != (
                    len(options_dict) + 1):  # +1 for one pmt history file
                time.sleep(5)
                print('waiting for all csv downloads to start')
                continue
            else:
                files = os.listdir(download_path)
                k = 0
                time.sleep(5)
                print('checking/waiting for all csv downloads to finish')
                for filename in files:
                    if 'crdownload' in filename:
                        print('{0} is still downloading'.format(filename))
                        time.sleep(30)
                    else:
                        k += 1
                        #                 print(k)
                if k == len(files):
                    time.sleep(2)
                    break

        print('done downloading')
        driver.close()
        return True
    except Exception as e:
        print(e)
        now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        driver.get_screenshot_as_file('screenshot-%s.png' % now)


def get_hashes(path):
    '''
    gets shasum hashes for files to check for file changes
    '''
    print('computing shasum for files in {0}'.format(os.path.abspath(path)))
    hashes = {}
    files = os.listdir(path)
    for file_ in files:
        sha = subprocess.check_output('shasum -a 256 {0}\
                                      '.format(path + '/'
                                               + file_),
                                      shell=True)
        hashes[file_] = sha.split()[0]
    return hashes


def check_file_changes(compare_dir, just_dled_hashes):
    '''
    Checks for sha differences for downloaded vs archived csvs
    '''
    need_to_clean = False
    print(
        'starting to check for file changes comparing what was just downloaded.'
    )
    print('looking for previous downloads at {0}'.format(compare_dir))
    try:
        previous_dled_hashes = get_hashes(compare_dir[1])

        # compare new download to previous download
        # check for added or deleted files
        dne_files = set(just_dled_hashes.keys()).intersection(
            set(previous_dled_hashes.keys()))
        add_files = set(previous_dled_hashes.keys()).intersection(
            set(just_dled_hashes.keys()))
        if len(just_dled_hashes) != len(previous_dled_hashes):
            need_to_clean = True
            print("Compared to the previous time new csv's were downloaded, \
                      the following files were deleted: \n {0}".format(
                          dne_files))
            print("Compared to the previous time new csv's were downloaded, \
                      the following files are new additions: \n {0}".format(
                          add_files))
        else:
            print(
                'No files were added or deleted since previous downloading of csvs'
            )

        # check for shasum256 changes
        changed_files = []
        for key in just_dled_hashes.keys() & previous_dled_hashes.keys():
            if previous_dled_hashes[key] != just_dled_hashes[key]:
                changed_files.append(key)

        if len(changed_files) == 0:
            print('There are no changes to previous downloaded lending club \
                      csvs (loan_info and pmt_hist) via shasum256 hashes')
        else:
            need_to_clean = True
            print('Compared to the previous data download, the shasum256 \
            hashes changed for the following files: {0}'.format(changed_files))

    except IndexError:
        need_to_clean = True
        print('Could not find previously download directory? This is \
                  probably your first time downloading the csvs or the \
                  first download to a new path.')

    return need_to_clean


def get_newest_creationtime_dir(ppath):
    '''returns path of newest dir by creation time in ppath'''
    print('getting folders sorted by creation time in {0}'.format(
        os.path.abspath(ppath)))
    csv_folders = [
        os.path.join(ppath, fn) for fn in os.listdir(ppath)
        if fn not in ['archived_csvs', 'working_csvs', 'latest_csvs']
    ]
    csv_folders = [(os.stat(path), path) for path in csv_folders]
    csv_folders = [(stat[ST_CTIME], path) for stat, path in csv_folders
                   if S_ISDIR(stat[ST_MODE])]
    return sorted(csv_folders)[-1]


# def archiver(archive_flag, ppath, archiver_dir=None):
#     '''will archive dirs if told to'''
#     archiver_dir = os.path.join(
#         os.path.expanduser('~'), 'projects', 'lendingclub', 'data', 'csvs',
#         'archived_csvs') if not archiver_dir else archiver_dir
#     os.makedirs(archiver_dir, exist_ok=True)
#     if archive_flag:
#         just_dled = get_sorted_creationtime_dirs(ppath)[-1][1]

#         newest_folder = os.path.split(just_dled)[1]

#         copytree(just_dled, os.path.join(archiver_dir, newest_folder))

#         print('copied {0} to {1}'.format(newest_folder,
#                                          os.path.abspath(archiver_dir)))


# def cleaner(ppath):
#     '''
#     cleans the ppath of every dir except archived_csvs
#     '''
#     just_dled = os.path.split(get_sorted_creationtime_dirs(ppath)[-1][1])[1]
#     keep_dirs = ['archived_csvs']  # , 'working_csvs', just_dled]
#     for tree in os.listdir(ppath):
#         if tree not in keep_dirs:
#             rmtree(os.path.join(ppath, tree), onerror=handle_error)
#             print('removing old dirs {0}'.format(tree))
#     return just_dled




# def handle_error(func, path, exc_info):
#     '''
#     Error handler function
#     It will try to change file permission and call the calling function again,
#     From https://thispointer.com/python-how-to-delete-a-directory-recursively-using-shutil-rmtree/
#     '''
#     print('Handling Error for file ', path)
#     print(exc_info)
#     # Check if file access issue
#     if not os.access(path, os.W_OK):
#         print('Hello')
#         # Try to change the permision of file
#         os.chmod(path, stat.S_IWUSR)
#         # call the calling function again
#         func(path)

Overwriting ../../lendingclub/csv_dl_archiving/download_prep.py


In [23]:
pause_len = 2500

# setup constants
email = acc_info.email_throwaway
password = acc_info.password_throwaway
url_dl = "https://www.lendingclub.com/info/download-data.action"
url_signin = "https://www.lendingclub.com/auth/login"
url_pmt_hist = "https://www.lendingclub.com/site/additional-statistics"

d_builder = DriverBuilder()
driver = d_builder.get_driver(download_location=download_path,
                              headless=True)

/home/justin/anaconda3/envs/lendingclub/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: use options instead of chrome_options


response from browser:
result:sessionId:36b8557cc299a97a56594d8a39ab9c37
result:status:0
result:value:None


In [24]:
# sign in
driver.get(url_signin)
pause.milliseconds(pause_len)
email_box = driver.find_element_by_name('email')
password_box = driver.find_element_by_name('password')
# password_box = driver.find_element_by_xpath(
#     '/html/body/div[2]/div[1]/div[2]/form[1]/label[2]/input')

pause.milliseconds(pause_len)
email_box.send_keys(email)
password_box.send_keys(password)

# button = driver.find_element_by_xpath(
#     '/html/body/div[2]/div[1]/div[2]/form[1]/button')
button = driver.find_element_by_class_name('form-button--submit')
button.click()
pause.milliseconds(pause_len)

driver.get(url_pmt_hist)

In [26]:
pmt_history = driver.find_element_by_partial_link_text('All payments')
pmt_history.click()
    
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
driver.get_screenshot_as_file('screenshot-%s.png' % now)

True

In [27]:
password

'1234567a'

# Below was for storing in s3 bucket

In [ ]:
aws_access_key_id = 
aws_secret_access_key = 

In [ ]:
import boto3

s3 = boto3.resource('s3')

In [ ]:
dirs = [d for d in os.listdir(ppath) if os.path.isdir(os.path.join(ppath,d))]
dirs.sort()
most_recent_dls = dirs[-1]

In [ ]:
files_to_store = os.listdir(ppath+'/'+most_recent_dls)

In [ ]:
ppath+'/'+most_recent_dls

In [ ]:
bucket_name = ''
for file_ in tqdm_notebook(files_to_store):
    data = open('{0}'.format(ppath+'/'+most_recent_dls+'/'+file_), 'rb')
    s3.Bucket(bucket_name).put_object(Key='{0}/{1}'.format(most_recent_dls, file_), Body=data)